This notebook allows you to calculate the potential costs of coastal flooding on buildings based on water depth. The calculation uses a vulnerability curve and costs that depend on the country and are refined based on the municipality's GDP. To do this, the user enters the link to their flood map raster as well as the modeling conditions (RP, year, SSP, and defense level).

# Activation of the necessary functions

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio    
import sys
sys.path.append('/path/to/gdal')  
import gc
from shapely.geometry import shape
gc.collect()
from shapely.validation import make_valid
import gcsfs
from exactextract import exact_extract
import warnings
import requests
warnings.filterwarnings("ignore")
import tempfile
from rasterio.enums import Resampling
from rasterio import features
from rasterio.warp import calculate_default_transform, reproject, Resampling
from branca.colormap import LinearColormap

## request information from the user

In [ ]:
pathflood=# PATH to the coastal flooding raster file, The file must be in .tif format (GeoTIFF).
year = "2010"# The possible values are 2010 , 2030, 2050, 2100
rp="100"# The possible values are static, 1, 100 or 1000
defense="UNDEFENDED" # The possible values are HIGH_DEFENDED, LOW_DEFENDED or UNDEFENDED
ssp="SSP126" # The possible values are SSP126, SSP245 or SSP585


In [ ]:
def validate_inputs(year, rp, defense, ssp):
    valid_years = {"2010", "2030", "2050", "2100"}
    valid_rps = {"static", "1", "100", "1000"}
    valid_defenses = {"HIGH_DEFENDED", "LOW_DEFENDED", "UNDEFENDED"}
    valid_ssps = {"SSP126", "SSP245", "SSP585"}
    errors = []
    if year not in valid_years:
        errors.append(f"Invalid year: {year}. Must be one of {sorted(valid_years)}.")
    if rp not in valid_rps:
        errors.append(f"Invalid return period (rp): {rp}. Must be one of {sorted(valid_rps)}.")
    if defense not in valid_defenses:
        errors.append(f"Invalid defense level: {defense}. Must be one of {sorted(valid_defenses)}.")
    if ssp not in valid_ssps:
        errors.append(f"Invalid SSP scenario: {ssp}. Must be one of {sorted(valid_ssps)}.")
    if errors:
        raise ValueError("\n".join(errors))
validate_inputs(year, rp, defense, ssp)

# CoCliCo connection and data recovery

In [ ]:
path = "https://storage.googleapis.com/coclico-data-public/coclico/bc_stats/bc_stats.parquet"
local_path = "CoCliCo_data/cost.parquet"
response = requests.get(path)
with open(local_path, "wb") as f:
    f.write(response.content)
path = "gs://coclico-data-public/coclico/LAU/LAU_2020_NUTS_2021_01M_3035.parquet"
local_path = "CoCliCo_data/LAU.parquet"
fs = gcsfs.GCSFileSystem()
fs.get(path, local_path)


## list of functions

In [ ]:
def modify_raster(input_raster_path): 
    with rasterio.open(input_raster_path) as src:
        raster_data = src.read(1) 
        nodata_value = src.nodata
        modified_data = np.full(raster_data.shape, np.nan, dtype=rasterio.float32)
        if nodata_value is not None:
            valid_mask = raster_data != nodata_value
            modified_data[valid_mask] = 1
        else:
            valid_mask = ~np.isnan(raster_data)
            modified_data[valid_mask] = 1
        return modified_data


def raster_value_to_shapefile(raster_path, target_value=1):
    with rasterio.open(raster_path) as src:
        raster_data = src.read(1)
        transform = src.transform
        crs = src.crs
        nodata = src.nodata
        shapes_and_values = rasterio.features.shapes(raster_data, transform=transform)
        geometries = []
        for geom, value in shapes_and_values:
            if value == target_value:  
                geometries.append({
                    'geometry': shape(geom),
                    'properties': {'value': value}
                })
        
        gdf = gpd.GeoDataFrame(geometries, crs=crs)
        return gdf

def reproject_raster(input_raster, target_crs="EPSG:3035"):
    with rasterio.open(input_raster) as src:
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        output_raster = input_raster.replace(".tif", "_3035.tif")
        with rasterio.open(output_raster, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest
                )
        return output_raster



def buildingvulnerabilite(df):
    df["codevulnerabilite"]="F22"
    return df

def _overlay_raster_vector(hazard_raster, features, hazard_crs, nodata=-9999, gridded=False, disable_progress=False):
    if hazard_raster.dtypes[0] == 'float64':
        hazard_raster = hazard_raster.read(1).astype(np.float32)
    area_and_line_objects = features.geom_type.isin(["Polygon", "MultiPolygon", "LineString", "MultiLineString"])
    point_objects = features.geom_type == "Point"
    if not gridded:
        if area_and_line_objects.sum() > 0:
            values_and_coverage_per_area_and_line_object = exact_extract(
                hazard_raster, 
                features[area_and_line_objects],
                ["coverage", "values"],
                output="pandas",
            )
            if not values_and_coverage_per_area_and_line_object.empty:
                values_and_coverage_per_area_and_line_object["h"] = values_and_coverage_per_area_and_line_object["values"].apply(np.mean)
                values_and_coverage_per_area_and_line_object["coverage"] = values_and_coverage_per_area_and_line_object["coverage"].apply(np.sum)
                features.loc[area_and_line_objects, "coverage"] = values_and_coverage_per_area_and_line_object["coverage"].values
                features.loc[area_and_line_objects, "h"] = values_and_coverage_per_area_and_line_object["h"].values
            else:
                features["h"] = np.nan 
    if "h" in features.columns:
        features = features[features["h"].apply(lambda x: not np.isnan(x))]
    return features


def calculdecout(df,surface,dommage,prix,nom):
    df[surface] = pd.to_numeric(df[surface], errors='coerce')
    df[prix] = pd.to_numeric(df[prix], errors='coerce')
    df[dommage] = pd.to_numeric(df[dommage], errors='coerce')

    df[nom]=df[surface]*df[prix]*df[dommage]
    return df

def save_modified_raster(modified_data, transform, crs, output_path):
    with rasterio.open(output_path, 'w', driver='GTiff', 
                       height=modified_data.shape[0], width=modified_data.shape[1],
                       count=1, dtype=modified_data.dtype, crs=crs, transform=transform) as dst:
        dst.write(modified_data, 1)

def courbedommage(df1,df2):
    df1['dommage'] = pd.NA 
    for index, row in df1.iterrows():
        h_value = row['h']
        codevulne = row['codevulnerabilite']
        if codevulne not in df2.columns:
            continue
        df2_row = df2[df2['h'] == h_value]
        if not df2_row.empty:
            dommage_value = df2_row[codevulne].values[0] 
            df1.at[index, 'dommage'] = dommage_value
    return df1


## link and download of files

### the data is downloaded and stored in the CoCliCo data folder, (infrastructure, vulnerability curve, cost, geographical unit and CoCliCo cost data)

In [ ]:

unitgeo="CoCliCo_data/LAU.parquet"
pathcurve="https://github.com/VincentBASCOUL/wb/blob/main/courbevulne.xlsx"
pathprice="https://github.com/VincentBASCOUL/wb/blob/main/cout.xlsx"
path_ratio="https://drive.google.com/uc?export=download&id=1ajGuXdyaA8sq5VUnJz8gpHmb2wguqSZm"
path_price2="https://github.com/VincentBASCOUL/wb/blob/main/coutparpays.xlsx"
response = requests.get(pathcurve)
with open("CoCliCo_data/vulnerability_curve.xlsx", "wb") as f:
    f.write(response.content)
response = requests.get(path_ratio)
with open("CoCliCo_data/gdp_ratio.gpkg", "wb") as f:
    f.write(response.content)
response = requests.get(path_price2)
with open("CoCliCo_data/price.xlsx", "wb") as f:
    f.write(response.content)
    

opening files, converting flood maps to vector and detecting the region in which it is located

In [ ]:
lau=gpd.read_parquet(unitgeo)
ratio=gpd.read_file("CoCliCo_data/gdp_ratio.gpkg")          
courbe=pd.read_excel("CoCliCo_data/courbevulne.xlsx", sheet_name='Feuil1')
pricebuild=pd.read_excel("CoCliCo_data/coutparpays.xlsx", sheet_name='Feuil1')
pathflood= reproject_raster(pathflood)
flood = modify_raster(pathflood)
with tempfile.NamedTemporaryFile(delete=False, suffix=".tif") as tmpfile:
    temp_raster_path = tmpfile.name
    with rasterio.open(pathflood) as src:
        save_modified_raster(flood, src.transform, src.crs, temp_raster_path)
        pixel_size_x, pixel_size_y = src.res 
    floodshp = raster_value_to_shapefile(temp_raster_path)
intersection= gpd.overlay(lau, floodshp, how="intersection")
unique_values = intersection['nuts_2'].unique()


## download infrastructure

In [ ]:
for i in range (len(unique_values)):
    path = "gs://coclico-data-public/coclico/ceed/"
    path=path+unique_values[i]+"_CEED.parquet"
    local_path = "CoCliCo_data/CEED/"+unique_values[i]+"_CEED.parquet"
    fs = gcsfs.GCSFileSystem()
    fs.get(path, local_path)

Calculation of the cost of potential damage based on water height

In [ ]:
stock2=gpd.GeoDataFrame()
for i in range (len(unique_values)):
    local_path = "CoCliCo_data/CEED/"+unique_values[i]+"_CEED.parquet"
    exposure=gpd.read_parquet(local_path)
    exposure.reset_index(inplace=True)            
    exposure=exposure[exposure["level_0"]=="buildings"]
    exposure = exposure.drop_duplicates(subset=['geometry'])
    exposure["areacal"]=exposure.area
    if 'level_0' not in exposure.columns:
        exposure.reset_index(drop=False, inplace=True)
    if 'level_0' in exposure.index.names:
        exposure['level_0'] = exposure.index.get_level_values('level_0')
        exposure.reset_index(drop=True, inplace=True) 
    exposure.drop(columns=['level_0'], inplace=True)
    exposure=gpd.sjoin(exposure,intersection,how="inner",predicate="intersects")# only building on flood
    exposure['geometry'] = exposure['geometry'].apply(lambda geom: make_valid(geom) if not geom.is_valid else geom)
    exposure = exposure[exposure['geometry'].is_valid]
    exposure = exposure.drop_duplicates(subset=['geometry'])
    if 'level_1' in exposure.columns:
        exposure.drop(columns=['level_1'], inplace=True)       
    if not exposure.empty:  
        exposure = buildingvulnerabilite(exposure)       
        with rasterio.open(pathflood) as hazard_raster:
            exposure = _overlay_raster_vector(hazard_raster=hazard_raster,features=exposure,hazard_crs=3035, nodata=-9999, gridded=False,disable_progress=False) 
        if 'level_1' in exposure.columns:
            exposure.drop(columns=['level_1'], inplace=True)                        
        if 'h' in exposure.columns: 
            exposure['h'] = pd.to_numeric(exposure['h'], errors='coerce')
            exposure['h'] = exposure['h'].astype('float64')
            exposure["h"]=exposure["h"].round(1)
            exposure=courbedommage(exposure, courbe)
            exposure["price"]=247.45592
            if 'coverage' in exposure.columns:
                exposure['coverage'].fillna(0, inplace=True)
                try:
                    exposure['areacal'] = exposure['coverage'] * abs(pixel_size_x)* abs(pixel_size_y)
                    
                except Exception as e:
                    print("Error ")
        exposure=calculdecout(exposure, "areacal", "dommage", "price","cost")
        stock2=gpd.GeoDataFrame(pd.concat([stock2, exposure], ignore_index=True))
pivot_table =stock2.groupby("GISCO_ID", as_index=False).sum(numeric_only=True)
pivot_table=pivot_table[["GISCO_ID","cost"]]       
        


compare the results with those of the platform

In [ ]:
column_j = f"{defense}_MAPS\\{rp}\\{ssp}\\{year}\\buildings"
coclico=gpd.read_parquet("CoCliCo_data/cost.parquet")
coclico[["GISCO_ID", column_j]]
merged = pd.merge(pivot_table, coclico[["GISCO_ID", column_j]], on="GISCO_ID", how='left')
merged=merged.rename(columns={column_j: "coclico"})
print(merged)
join_column_gdf = "GISCO_ID"
join_column_df ="GISCO_ID"
merged=lau.merge(merged,left_on=join_column_gdf, right_on=join_column_df, how="inner")

vmin =merged["cost"].min()
vmax = merged["cost"].max()
steps = np.linspace(vmin, vmax, 5)
colormap = LinearColormap(
    colors=["white", "red"],
    vmin=vmin,
    vmax=vmax
).to_step(index=steps)
merged.explore(column="cost",  # Colorer selon une colonne
    cmap=colormap,
    legend=True,
    legend_kwds={"position": "bottomright", "orientation": "vertical"},

    tooltip=["LAU_NAME","GISCO_ID", "cost", "coclico"],  # Infos au survol
    style_kwds={"fillOpacity": 1.0, "color": "black", "weight": 0.5})


